# Data Transformation using Pandas

S.Yu. Papulin (papulin.study@yandex.ru)

### Contents
 - [Preparation](#Preparation)
 - [Basic operations over dataframes](#Basic-operations-over-dataframes)
     - [Selecting and filtering](#Selecting-and-filtering)
     - [Grouping](#Grouping)
 - [Applying user-defined function](#Applying-user-defined-function)
     - [Extracting place locations](#Extracting-place-locations)
     - [Calculating distance](#Calculating-distance)
     - [Matching places to districts](#Matching-places-to-districts)
 - [Plotting data onto map](#Plotting-data-onto-map)

## Preparation

Import modules

In [1]:
import pandas as pd
import numpy as np

In [2]:
# !pip install --upgrade pandas==0.23

**Installing some additional modules if needed**

⚠️ To use `GeoDataFrame`, please, install `geopandas` as it's not a part of the `anaconda` distribution

In [3]:
# Module for processing geodata as DataFrames
# !pip install geopandas==0.8

In [4]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geopandas.tools import sjoin

⚠️ To use `sjoin`, please, install `rtree` as it's not a part of the `anaconda` distribution

In [5]:
# !pip install rtree

⚠️ To use maps, please, install `folium` as it's not a part of the `anaconda` distribution

In [6]:
# Module for map visualization
# !pip install folium

In [7]:
import folium
from folium.plugins import HeatMap

⚠️ There is a bug when displaying a map in Chrome. To get around this issue, use the code below to display maps

In [8]:
# https://github.com/python-visualization/folium/issues/812
def embed_map(m):
    from IPython.display import IFrame
    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

## Basic operations over dataframes

In [ ]:
FILE = "../data/data-4275-2021-02-09.xlsx"

In [ ]:
CLMNS = ["Name", "TypeObject", "District", "Address", "SeatsCount", "geoData"]

In [ ]:
# Create a dataframe of places
df = pd.read_excel(FILE, index_col=0)
df.head()

In [ ]:
# Columns in lowercase
# df.columns = list(map(str.lower, df.columns))

In [ ]:
# Data description
df.info()

In [ ]:
# Select only needed columns
df_places = df[CLMNS]
df_places.head()

In [ ]:
# Data description
df_places.info()

In [ ]:
# Alternative way using read_excel
df_places = pd.read_excel(FILE, 
                          index_col=0, 
                          usecols=["ID",] + CLMNS)  # or you can use indices [0, 1, 5, 7, 8, 10, 14]
df_places.head()

Print types of public catering

In [ ]:
df_places["TypeObject"].unique()

Find places on Bauman street:

In [ ]:
cond = df_places["Address"].str.lower().str.contains("бауманск")

In [ ]:
df_baum = df_places[cond]
df_baum.head()

In [ ]:
# query expression
expr = 'Address.str.lower().str.contains("бауманс")'

# filter rows
df_places.query(expr, engine='python').head()

### Selecting and filtering

In [ ]:
CLMNS = ["Name", "Address"]
QUERY = "SeatsCount == 35"

In [ ]:
# Option 1: Using loc
df_places.loc[
    df_places["SeatsCount"] == 35,  # filter
    CLMNS  # select columns
].head()

In [ ]:
# Option 2: Using query
df_places.query(QUERY)[CLMNS].head()

In [ ]:
# Option 3: Using filter
df_places\
    .filter(items=CLMNS, axis=1)\
    .filter(items=df_places.index[df_places["SeatsCount"] == 35], axis=0)\
    .head()

In [ ]:
# Option 4(a): Using pipe
df_places\
    .query(QUERY)\
    .pipe(lambda df: df[CLMNS])\
    .head()

In [ ]:
# Option 4(b): Using pipe
def select_columns(df, clmns):
    return df[clmns]

def filter_rows(df, condition):
    return df.query(condition)

In [ ]:
df_places_new = df_places\
    .pipe(filter_rows, condition=QUERY)\
    .pipe(select_columns, clmns=CLMNS)
df_places_new.head()

In [ ]:
# Alternative way
df_places_new = (
    df_places
    .pipe(filter_rows, condition=QUERY)  # you can provide comments here
    .pipe(select_columns, clmns=CLMNS)
)
df_places_new.head()

### Grouping

In [ ]:
# Option 1
df_places__counts = df_places[["TypeObject"]]\
    .groupby(["TypeObject"])\
    .size()\
    .reset_index(name="Counts")\
    .sort_values(["Counts",], ascending=False)
df_places__counts

In [ ]:
# Option 2
df_places__counts = df_places[["TypeObject"]]\
    .groupby(["TypeObject"])["TypeObject"]\
    .count()\
    .reset_index(name="Counts")\
    .sort_values(["Counts",], ascending=False)
df_places__counts

In [ ]:
# Option 3. min, max, sum, mean etc.
df_places__counts = df_places[["TypeObject"]]\
    .groupby(["TypeObject"])\
    .agg({"TypeObject": "count"})\
    .rename(columns={"TypeObject": "Counts"})\
    .reset_index(level="TypeObject")\
    .sort_values(["Counts",], ascending=False)
df_places__counts

In [ ]:
# Option 4
df_places__counts = df_places["TypeObject"]\
    .value_counts()\
    .reset_index(name="TypeObject")
df_places__counts

In [ ]:
# Option 5 (>=  Pandas 1.1.0)
# df_places.value_counts(subset=["TypeObject",])

Print out top-5 places for each type by their number of seats in descending order

In [ ]:
df_places.sort_values("SeatsCount", ascending=[0])\
    .groupby("TypeObject")\
    .head(5)

In [ ]:
# Grouped output
df_places.sort_values(["TypeObject", "SeatsCount"], ascending=[1, 0])\
    .groupby("TypeObject")\
    .head(5)

## Applying user-defined function

### Extracting place locations

Print out a single line of the geoData column

In [ ]:
df_places.iloc[0]["geoData"]

In [ ]:
import re
import json

In [ ]:
def extract_coordinates(geo_data):
    """Parse geoData value."""
    try:
        geo_data_ = re.sub("(\w+)=(\w+), (\w+)=", r'"\1": "\2", "\3": ', geo_data)
        return json.loads(geo_data_)["coordinates"]
    except:
        pass

In [ ]:
# Create a new dataframe with lat, lng columns
df_places__coords = df_places.apply(
    lambda x: pd.Series(
        extract_coordinates(x["geoData"]), 
        index=["Lng", "Lat"]
    ),
    axis=1, 
    result_type="expand"
)

df_places__coords.head()

In [ ]:
# Option 1: Merge the initial dataframe and the dataframe with coordinates
df_places_ext = df_places.merge(df_places__coords, left_index=True, right_index=True)
df_places_ext.head()

In [ ]:
# Option 2: Concatenate the initial dataframe and the dataframe with coordinates
df_places_ext = pd.concat([df_places, df_places__coords], axis=1, sort=False)
df_places_ext.head()

### Calculating distance

Find your location by your ip address

In [ ]:
import requests

In [ ]:
# URL for getting your public IP
IP_SERVICE_URL = "https://api.myip.com"
IP_SERVICE_URL = "https://api.ipify.org/?format=json"

# URL for getting your location by your IP
LOCATION_SERVICE_URL = "http://ip-api.com/json/{}"

Getting your location by IP

In [ ]:
# Request your public IP
ip__response = requests.get(IP_SERVICE_URL)
ip__response__data = ip__response.json()
ip = ip__response__data["ip"]

# Request your location
location__response = requests.get(LOCATION_SERVICE_URL.format(ip))
location__response__data = location__response.json()

# Extract lat and lng
location = location__response__data["lat"], location__response__data["lon"]
location

Computing distances between your location and places

In [ ]:
def calculate_haversine(*coords):
    """
    Calculate a distance between points.
    
    Note: (start lat, start lng, end lat, end lng)
    """
    if len(coords) != 4:
        return
    radius = 6371.0
    coords_radians = np.radians(coords)
    return 2 * radius * np.arcsin(
        np.sqrt(
            np.sin((coords_radians[2]-coords_radians[0])/2.0)**2 +
            np.cos(coords_radians[0])*np.cos(coords_radians[2])*np.sin(
                (coords_radians[3]-coords_radians[1])/2.0)**2
        )
    )

In [ ]:
s_distance = df_places_ext\
    .apply(
        lambda row: calculate_haversine(
            location[0], 
            location[1], 
            row["Lat"], 
            row["Lng"]), axis=1)\
    .rename("Distance")

s_distance.head()

In [ ]:
# Add the distance column
df_places_ext_ = df_places_ext.merge(s_distance, left_index=True, right_index=True)
df_places_ext_.head()

In [ ]:
# df_ext_ = pd.concat([df_ext, s_distance], axis=1, sort=False)
# df_ext_.head()

In [ ]:
# Places within 3km radius
df_less_3km = df_places_ext_.query("Distance < 3")
df_less_3km.head()

In [ ]:
# 5 nearest places to your location
df_less_3km\
    .sort_values("Distance")\
    .head(5)

### Matching places to districts

In [ ]:
# Path where city geojson is located
MOSCOW_DISTRICTS_PATH = "../data/mo.geojson"

In [ ]:
with open(MOSCOW_DISTRICTS_PATH, encoding="utf-8") as f:
    districts_geojson = json.load(f)

In [ ]:
# Single zone
districts_geojson["features"][0]

In [ ]:
sgeometry_type = set()
for district in districts_geojson["features"]:
    sgeometry_type.add(district["geometry"]["type"])
sgeometry_type

**Using `shapely`**

In [ ]:
from shapely.geometry import Point, Polygon, MultiPolygon

In [ ]:
def create_district_row(features):
    """
    Create a district row with properties and geometry field of 
    the Polygon/MultiPolygon type.
    """
    for item in features:
        try:
            row = list(item["properties"].values()).copy()    
            polygons = list()
            if item["geometry"]["type"] == "Polygon":
                polygons.append(Polygon(item["geometry"]["coordinates"][0]))
            elif item["geometry"]["type"] == "MultiPolygon":
                for polygon in item["geometry"]["coordinates"]:
                    polygons.append(Polygon(polygon[0]))
            row.append(MultiPolygon(polygons=polygons))
            yield row
        except Exception as e:
            pass

In [ ]:
# Test the function using a single zone
list(create_district_row([districts_geojson["features"][0],]))

In [ ]:
# Extract column names for districts
column_name_list = [key.title() for key, value in districts_geojson["features"][0]["properties"].items()]
column_name_list += ["geometry"]
column_name_list

In [ ]:
# Create a dataframe of districts
df_districts = pd.DataFrame(data=create_district_row(districts_geojson["features"]), 
                            columns=column_name_list)
df_districts.set_index("Oktmo", inplace=True)
df_districts.head()

In [ ]:
# Create the geometry field for places
df_places_ext["geometry"] = df_places_ext\
    .apply(lambda row: Point((row["Lng"], row["Lat"])), axis=1)
df_places_ext.head()

Mapping places to districts

⏱️ It takes some time...

In [ ]:
# %%timeit -n1 -r1
# Option 1
df_districts["Counts"] = 0
for place_indx, place in df_places_ext.iterrows():
    for district_indx, district in df_districts.iterrows():
        if district["geometry"].contains(place["geometry"]): 
            df_districts.loc[district_indx, "Counts"] += 1
            break

In [ ]:
df_districts.head()

In [ ]:
# Option 2
def place_in(districts):
    """Return a district index for a place."""
    def _place_in(place):
        for index, district in districts.iterrows():
            if district["geometry"].contains(place["geometry"]):
                return index
    return _place_in

⏱️ It takes some time...

In [ ]:
# %%timeit -n1 -r1
# Add a column with district indices
df_places_ext["Oktmo"] = df_places_ext.apply(place_in(df_districts), axis=1)
df_places_ext.head()

In [ ]:
# Add a column with the number of places for each district
df_districts["Counts"] = df_places_ext\
    .groupby(["Oktmo",])\
    .size()
df_districts.head()

**Using `geopandas`**

In [ ]:
# Note: Remove the Oktmo column or just select columns that needed
df_places_ext.drop(["Oktmo",], axis=1, inplace=True, errors="ignore")

In [ ]:
# Convert the place DataFrame to GeoDataFrame
geodf_places = gpd.GeoDataFrame(df_places_ext, geometry=df_places_ext["geometry"])
geodf_places.head()

In [ ]:
# Convert the district DataFrame to GeoDataFrame
geodf_districts = gpd.GeoDataFrame(df_districts, geometry=df_districts["geometry"])
geodf_districts.head()

In [ ]:
# %%timeit -n1
# Count the number of places for each district
df_districts["Counts"] = sjoin(geodf_districts, geodf_places, how="left", op="contains")\
        .groupby(["Oktmo"])\
        .size()
df_districts.head(5)

## Plotting data onto map

**Plotting city districts**

In [ ]:
with open(MOSCOW_DISTRICTS_PATH, encoding="utf-8") as f:
    districts_geojson = json.load(f)

In [ ]:
# Create an instance of map
m = folium.Map()

In [ ]:
# Plot zones onto map
style_function = lambda x: {
    "color" : "orange",
    "weight": 1
}

folium\
    .GeoJson(districts_geojson, name="geojson", style_function=style_function)\
    .add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

**Plotting places**

In [ ]:
# Columns to use
CLMNS = ["Name", "Lat", "Lng"]

In [ ]:
df_places__coords = df_places_ext[CLMNS]
df_places__coords.head()

In [ ]:
# Create markers for places
for indx, row in df_places__coords.iterrows():
    try:
        folium.Circle(
            radius=5,
            location=[row["Lat"], row["Lng"]],
            popup=row["Name"],
            color="red",
            fill=True,
            fill_opacity=1.0
        ).add_to(m)
    except:
        pass

In [ ]:
# Create a marker for your location
folium.Marker(location, icon=folium.Icon(color="darkblue", 
                                         icon_color="white", 
                                         prefix="fa", 
                                         icon="user")).add_to(m)

In [ ]:
# Plot map with all elements
embed_map(m)

**Plotting city districts and place counts**

In [ ]:
df_districts.reset_index(level="Oktmo").head()

In [ ]:
m = folium.Map()

folium.Choropleth(
    geo_data=districts_geojson,
    data=df_districts.reset_index(level="Oktmo"),  # index has to be a separate column
    columns=["Oktmo", "Counts",],
    key_on="feature.properties.OKTMO",
    name="Number of places",
    legend_name="Number of places",
    highlight=True,
    nan_fill_color="grey",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)